In [174]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from google.colab import files
import io

import imghdr
import os

print(tf.__version__)

2.2.0-rc2


In [175]:
tf.executing_eagerly()

True

# Using Drive Mounting Instead (Tutorial: https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92)

To mount Google Drive:
- Run the code below
- Click the URL 
- Sign in & click Allow to get the authorization code
- Copy the authorization code and paste it to:
  Enter your authorization code: 
  ____________

In [176]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


After mounting, Collab will be connected to your Google Drive
Click the File button on the left <-, click 'drive' then 'My Drive', what we need is:
- sample_labels.csv file and the ct_scan_img.zip
- If it is not there, you can copy it from the Bangkit Group Project Folder
- Bangkit Group Project (Assig #5)/DATASET/CT_SCAN/ct_scan_img.zip"

Getting the sample_labels.csv in Colab:
- Find the sample_labels.csv file in your drive folder, right-click, and click COPY PATH
- Change the path variable below with the path you copied

In [0]:
path = "/content/drive/My Drive/Bangkit Group Project (Assig #5)/DATASET/CT_SCAN/sample_labels.csv"
sample_labels_all = pd.read_csv(path)

In [178]:
sample_labels_all.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,5,13,060Y,M,AP,3056,2544,0.139,0.139
1,00000013_026.png,Cardiomegaly|Emphysema,26,13,057Y,M,AP,2500,2048,0.168,0.168
2,00000017_001.png,No Finding,1,17,077Y,M,AP,2500,2048,0.168,0.168
3,00000030_001.png,Atelectasis,1,30,079Y,M,PA,2992,2991,0.143,0.143
4,00000032_001.png,Cardiomegaly|Edema|Effusion,1,32,055Y,F,AP,2500,2048,0.168,0.168


We want each label to have 1 finding, therefore for those with multiple findings, we are picking one out of them according to our defined ranking.

In [0]:
#sample_labels = sample_labels_all.sample(5000, replace=False) #[:1000]

In [0]:
ranking = ["pleural_thickening", "effusion", "consolidation", "hernia", "pneumonia", "fibrosis", 
           "edema", "emphysema", "cardiomegaly", "pneumothorax", "mass", "nodule", 
           "atelectasis", "infiltration", "no finding"]
#[14, 4, 5, 6, 7, 8, 1, 2, 9, 10, ]
#["pleural_thickening", "consolidation", "effusion", "pneumonia", ]

In [0]:
def pick_one(label):
  splitted = label.split('|')
  print(splitted)
  lowered = [s.lower() for s in splitted]
  print(lowered)
  index_rank = [ranking.index(word) for word in lowered]
  print(index_rank)
  min_rank = min(index_rank)
  print(min_rank)
  picked = ranking[min_rank]
  return picked
  #index_found = [options.index(word) for word in lowered if word in options]
  #result = index_found

  #return result

In [182]:
result = pick_one('Emphysema|Infiltration|Pleural_Thickening|Pneumothorax')
result

['Emphysema', 'Infiltration', 'Pleural_Thickening', 'Pneumothorax']
['emphysema', 'infiltration', 'pleural_thickening', 'pneumothorax']
[7, 13, 0, 9]
0


'pleural_thickening'

In [183]:
init_labels = sample_labels_all['Finding Labels'].tolist()
print(len(init_labels))
print(init_labels[:5])

5606
['Emphysema|Infiltration|Pleural_Thickening|Pneumothorax', 'Cardiomegaly|Emphysema', 'No Finding', 'Atelectasis', 'Cardiomegaly|Edema|Effusion']


In [184]:
picked_labels = [pick_one(label) for label in init_labels]

Streaming output truncated to the last 5000 lines.
['no finding']
[14]
14
['Edema', 'Effusion', 'Nodule']
['edema', 'effusion', 'nodule']
[6, 1, 11]
1
['No Finding']
['no finding']
[14]
14
['No Finding']
['no finding']
[14]
14
['Infiltration']
['infiltration']
[13]
13
['No Finding']
['no finding']
[14]
14
['No Finding']
['no finding']
[14]
14
['Atelectasis']
['atelectasis']
[12]
12
['No Finding']
['no finding']
[14]
14
['Infiltration']
['infiltration']
[13]
13
['No Finding']
['no finding']
[14]
14
['No Finding']
['no finding']
[14]
14
['No Finding']
['no finding']
[14]
14
['No Finding']
['no finding']
[14]
14
['No Finding']
['no finding']
[14]
14
['Infiltration']
['infiltration']
[13]
13
['No Finding']
['no finding']
[14]
14
['No Finding']
['no finding']
[14]
14
['No Finding']
['no finding']
[14]
14
['Atelectasis']
['atelectasis']
[12]
12
['Infiltration']
['infiltration']
[13]
13
['Pleural_Thickening']
['pleural_thickening']
[0]
0
['No Finding']
['no finding']
[14]
14
['No Finding']
['

In [185]:
picked_labels[:10]

['pleural_thickening',
 'emphysema',
 'no finding',
 'atelectasis',
 'effusion',
 'consolidation',
 'no finding',
 'no finding',
 'effusion',
 'no finding']

In [0]:
sample_labels_all['Old Labels'] = sample_labels_all['Finding Labels']
sample_labels_all['Finding Labels'] = picked_labels 

In [187]:
sample_labels_all.groupby('Finding Labels').count()

,Image Index,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y,Old Labels
Finding Labels,,,,,,,,,,,
atelectasis,249,249,249,249,249,249,249,249,249,249,249
cardiomegaly,66,66,66,66,66,66,66,66,66,66,66
consolidation,156,156,156,156,156,156,156,156,156,156,156
edema,76,76,76,76,76,76,76,76,76,76,76
effusion,606,606,606,606,606,606,606,606,606,606,606
emphysema,89,89,89,89,89,89,89,89,89,89,89
fibrosis,60,60,60,60,60,60,60,60,60,60,60
hernia,11,11,11,11,11,11,11,11,11,11,11
infiltration,503,503,503,503,503,503,503,503,503,503,503


In [0]:
average_num = sample_labels_all.groupby('Finding Labels').count()[['Image Index']].reset_index()

In [189]:
average_num

,Finding Labels,Image Index
0,atelectasis,249
1,cardiomegaly,66
2,consolidation,156
3,edema,76
4,effusion,606
5,emphysema,89
6,fibrosis,60
7,hernia,11
8,infiltration,503
9,mass,152


In [190]:
average_is = np.mean(average_num['Image Index'].tolist())
average_is

373.73333333333335

In [191]:
stad = np.std(average_num['Image Index'].tolist())
stad

731.4618665536996

### Balancing the Data

In [192]:
count = sample_labels_all.groupby('Finding Labels').count()[['Image Index']].reset_index()
count

,Finding Labels,Image Index
0,atelectasis,249
1,cardiomegaly,66
2,consolidation,156
3,edema,76
4,effusion,606
5,emphysema,89
6,fibrosis,60
7,hernia,11
8,infiltration,503
9,mass,152


In [193]:
above_100 = count[count['Image Index'] > 100]
above_100

,Finding Labels,Image Index
0,atelectasis,249
2,consolidation,156
4,effusion,606
8,infiltration,503
9,mass,152
10,no finding,3044
11,nodule,202
12,pleural_thickening,176
14,pneumothorax,172


In [194]:
above_100_without_nofinding = above_100[above_100['Finding Labels'] != 'no finding']
above_100_without_nofinding

,Finding Labels,Image Index
0,atelectasis,249
2,consolidation,156
4,effusion,606
8,infiltration,503
9,mass,152
11,nodule,202
12,pleural_thickening,176
14,pneumothorax,172


In [195]:
below_100 = count[count['Image Index'] < 100]
below_100

,Finding Labels,Image Index
1,cardiomegaly,66
3,edema,76
5,emphysema,89
6,fibrosis,60
7,hernia,11
13,pneumonia,44


In [0]:
sample_labels = sample_labels_all[sample_labels_all['Finding Labels'] != 'cardiomegaly']
sample_labels = sample_labels[sample_labels['Finding Labels'] != 'edema']
sample_labels = sample_labels[sample_labels['Finding Labels'] != 'emphysema']
sample_labels = sample_labels[sample_labels['Finding Labels'] != 'fibrosis']
sample_labels = sample_labels[sample_labels['Finding Labels'] != 'hernia']
sample_labels = sample_labels[sample_labels['Finding Labels'] != 'pneumonia']

In [197]:
# TESTING WITH NO -> NO FINDINGS
print(len(sample_labels))
sample_labels = sample_labels[sample_labels['Finding Labels'] != 'no finding']
print(len(sample_labels))

5260
2216


In [198]:
# Add back SOME no findings
average_label_num = int(np.mean(above_100_without_nofinding['Image Index'].tolist()))
print(average_label_num)

277


In [0]:
sample_no_finding = sample_labels_all[sample_labels_all['Finding Labels'] == 'no finding'].sample(average_label_num, replace=False)

In [200]:
sample_no_finding

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y,Old Labels
4671,00023451_000.png,no finding,0,23451,053Y,F,PA,2500,2048,0.168,0.168,No Finding
661,00003426_007.png,no finding,7,3426,058Y,M,AP,2500,2048,0.171,0.171,No Finding
5142,00027370_001.png,no finding,1,27370,036Y,M,PA,2992,2991,0.143,0.143,No Finding
2877,00014203_028.png,no finding,28,14203,031Y,M,AP,2048,2500,0.168,0.168,No Finding
964,00005026_000.png,no finding,0,5026,037Y,M,PA,2500,2048,0.171,0.171,No Finding
...,...,...,...,...,...,...,...,...,...,...,...,...
1175,00006249_003.png,no finding,3,6249,053Y,M,AP,2500,2048,0.171,0.171,No Finding
1353,00007034_039.png,no finding,39,7034,030Y,F,AP,3056,2544,0.139,0.139,No Finding
5095,00027075_001.png,no finding,1,27075,037Y,M,PA,2674,2601,0.143,0.143,No Finding
4007,00019766_004.png,no finding,4,19766,037Y,M,PA,2992,2991,0.143,0.143,No Finding


In [0]:
sample_labels_added = sample_labels.append(sample_no_finding)

In [202]:
print(len(sample_labels_added))
sample_labels_added.groupby('Finding Labels').count()

2493


,Image Index,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y,Old Labels
Finding Labels,,,,,,,,,,,
atelectasis,249,249,249,249,249,249,249,249,249,249,249
consolidation,156,156,156,156,156,156,156,156,156,156,156
effusion,606,606,606,606,606,606,606,606,606,606,606
infiltration,503,503,503,503,503,503,503,503,503,503,503
mass,152,152,152,152,152,152,152,152,152,152,152
no finding,277,277,277,277,277,277,277,277,277,277,277
nodule,202,202,202,202,202,202,202,202,202,202,202
pleural_thickening,176,176,176,176,176,176,176,176,176,176,176
pneumothorax,172,172,172,172,172,172,172,172,172,172,172


In [0]:
# TRY LARGER VALUES
sample_labels = sample_labels_added.sample(2480, replace=False) #500 TRY ALL cannot 2493

In [204]:
sample_labels.groupby('Finding Labels').count()

,Image Index,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y,Old Labels
Finding Labels,,,,,,,,,,,
atelectasis,247,247,247,247,247,247,247,247,247,247,247
consolidation,154,154,154,154,154,154,154,154,154,154,154
effusion,604,604,604,604,604,604,604,604,604,604,604
infiltration,501,501,501,501,501,501,501,501,501,501,501
mass,151,151,151,151,151,151,151,151,151,151,151
no finding,276,276,276,276,276,276,276,276,276,276,276
nodule,202,202,202,202,202,202,202,202,202,202,202
pleural_thickening,175,175,175,175,175,175,175,175,175,175,175
pneumothorax,170,170,170,170,170,170,170,170,170,170,170


In [205]:
labels = sample_labels['Finding Labels'].tolist()
len(labels)

2480

In [0]:
#above_100_labels = above_100['Finding Labels'].tolist()

In [0]:
#sample_labels = sample_labels_all[sample_labels_all['Finding Labels'] == above_100_labels[0]]

In [0]:
#[for label in above_100_labels]

Getting the CT Scan Images to Colab:
- Find the ct_scan_img.zip file in your drive folder (in Colab)
- Right-click, then click COPY PATH
- Paste the change the path I have, which is: "/content/drive/My Drive/Bangkit Group Project (Assig #5)/DATASET/CT_SCAN/ct_scan_img.zip" BELOW, with the one you have.
- And then run the !unzip to unzip the images

In [0]:
!unzip -uq "/content/drive/My Drive/Bangkit Group Project (Assig #5)/DATASET/CT_SCAN/ct_scan_img.zip"

From the Sample Labels dataframe / table, we get the filename of the images, and make a list out of that called 'image_dir' list.

In [0]:
image_dir = sample_labels['Image Index'].tolist()

In [211]:
# Just checking, carry on~
sample_labels[sample_labels['Image Index'] == image_dir[0]]['Finding Labels']

922    effusion
Name: Finding Labels, dtype: object

In [212]:
type(image_dir[0])

str

In [213]:
image_dir[:5]

['00004850_008.png',
 '00021499_013.png',
 '00014571_002.png',
 '00015236_000.png',
 '00021289_000.png']

The RAM available in Google Colab is LIMITED, I believe initially it gives 15 GB.
If there is an error of: Colab doesn't have enough RAM, usually it asks if we want MORE RAM, click yes!
And it will give us 35 GB.

But, it still won't be enough for us to try with ALL of the images.
Therefore, we will only use 100 Images.

In [214]:
test_dir = image_dir #[:100] # Taking the first 100 Images
len(test_dir)

2480

In [215]:
test_dir[:5]

['00004850_008.png',
 '00021499_013.png',
 '00014571_002.png',
 '00015236_000.png',
 '00021289_000.png']

Creating the Datasets:
The tf.data.Dataset API supports writing descriptive and efficient input pipelines. Dataset usage follows a common pattern:

- Create a source dataset from your input data.
- Apply dataset transformations to preprocess the data.
- Iterate over the dataset and process the elements.

Iteration happens in a streaming fashion, so the full dataset does not need to fit into memory.

Sources can be from:
- LIST, using -> from_tensor_slices
- Files given filename, using -> list_files

In [0]:
dataset_from_list = tf.data.Dataset.list_files(test_dir)

In [217]:
dataset_from_list

<ShuffleDataset shapes: (), types: tf.string>

In [218]:
for element in dataset_from_list:
  print(element)

tf.Tensor(b'./00016469_000.png', shape=(), dtype=string)
tf.Tensor(b'./00008008_068.png', shape=(), dtype=string)
tf.Tensor(b'./00000832_005.png', shape=(), dtype=string)
tf.Tensor(b'./00020935_004.png', shape=(), dtype=string)
tf.Tensor(b'./00006212_005.png', shape=(), dtype=string)
tf.Tensor(b'./00005200_001.png', shape=(), dtype=string)
tf.Tensor(b'./00021670_027.png', shape=(), dtype=string)
tf.Tensor(b'./00019899_025.png', shape=(), dtype=string)
tf.Tensor(b'./00002425_000.png', shape=(), dtype=string)
tf.Tensor(b'./00021648_006.png', shape=(), dtype=string)
tf.Tensor(b'./00019863_005.png', shape=(), dtype=string)
tf.Tensor(b'./00000938_004.png', shape=(), dtype=string)
tf.Tensor(b'./00004605_019.png', shape=(), dtype=string)
tf.Tensor(b'./00022596_008.png', shape=(), dtype=string)
tf.Tensor(b'./00020826_000.png', shape=(), dtype=string)
tf.Tensor(b'./00013670_036.png', shape=(), dtype=string)
tf.Tensor(b'./00002570_004.png', shape=(), dtype=string)
tf.Tensor(b'./00020209_005.png'

In [0]:
dataset_of_filenames = tf.data.Dataset.from_tensor_slices(test_dir) 

In [220]:
dataset_of_filenames

<TensorSliceDataset shapes: (), types: tf.string>

In [221]:
for element in dataset_of_filenames:
  print(element)

tf.Tensor(b'00004850_008.png', shape=(), dtype=string)
tf.Tensor(b'00021499_013.png', shape=(), dtype=string)
tf.Tensor(b'00014571_002.png', shape=(), dtype=string)
tf.Tensor(b'00015236_000.png', shape=(), dtype=string)
tf.Tensor(b'00021289_000.png', shape=(), dtype=string)
tf.Tensor(b'00015078_001.png', shape=(), dtype=string)
tf.Tensor(b'00025386_000.png', shape=(), dtype=string)
tf.Tensor(b'00028063_002.png', shape=(), dtype=string)
tf.Tensor(b'00001558_005.png', shape=(), dtype=string)
tf.Tensor(b'00007442_008.png', shape=(), dtype=string)
tf.Tensor(b'00022720_003.png', shape=(), dtype=string)
tf.Tensor(b'00021988_005.png', shape=(), dtype=string)
tf.Tensor(b'00004737_016.png', shape=(), dtype=string)
tf.Tensor(b'00026132_026.png', shape=(), dtype=string)
tf.Tensor(b'00022816_000.png', shape=(), dtype=string)
tf.Tensor(b'00022335_018.png', shape=(), dtype=string)
tf.Tensor(b'00013849_007.png', shape=(), dtype=string)
tf.Tensor(b'00007779_004.png', shape=(), dtype=string)
tf.Tensor(

As we can see, whether it is made using list_files or from_tensor_slices it outputs the same dataset. 

In [222]:
for f in dataset_of_filenames.take(1):
    print(f.numpy())
    print(f)
    print(type(f))
    filename_str = str(f.numpy())[2:-1]
    print(filename_str)
    print(type(filename_str))
    print(type(str(f.numpy())))
    print(image_dir[0])
    print(filename_str == image_dir[0])
    print(sample_labels[sample_labels['Image Index'] == filename_str].reset_index().iloc[0]['Finding Labels'])


b'00004850_008.png'
tf.Tensor(b'00004850_008.png', shape=(), dtype=string)
<class 'tensorflow.python.framework.ops.EagerTensor'>
00004850_008.png
<class 'str'>
<class 'str'>
00004850_008.png
True
effusion


In [223]:
def parse_data_without_augmentation(filename, IMAGE_SIZE): #, scores, IMAGE_SIZE):
    '''
    Loads the image file without any augmentation. Used for validation set.
    Args:
        filename: the filename from the record
        scores: the scores from the record
    Returns:
        an image referred to by the filename and its scores
    '''

    image = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    image = (tf.cast(image, tf.float32) - 127.5) / 127.5

    return image #, label #, scores
    
""""
    print("FILENAME")
  
    print(str(filename))
    print(tf.strings.as_string(filename))
    tf.compat.v1.enable_eager_execution()
    filename_str = str(filename.numpy())[2:-1]
    # sample_labels['Finding Labels'].tolist()
    label = sample_labels[sample_labels['Image Index'] == filename_str].reset_index().iloc[0]['Finding Labels']
    #sample_labels[sample_labels['Image Index'] == str(filename)]['Finding Labels']
    print(label)

"""""


'"\n    print("FILENAME")\n  \n    print(str(filename))\n    print(tf.strings.as_string(filename))\n    tf.compat.v1.enable_eager_execution()\n    filename_str = str(filename.numpy())[2:-1]\n    # sample_labels[\'Finding Labels\'].tolist()\n    label = sample_labels[sample_labels[\'Image Index\'] == filename_str].reset_index().iloc[0][\'Finding Labels\']\n    #sample_labels[sample_labels[\'Image Index\'] == str(filename)][\'Finding Labels\']\n    print(label)\n\n'

In [224]:
IMG_WIDTH = 160
IMG_HEIGHT = 160
IMG_SIZE = 160
IMAGE_SIZE = 160
SHUFFLE_BUFFER_SIZE = 1000
BATCH_SIZE = 25 #1 #####32
#print("BATCH SIZE: "+str(BATCH_SIZE))
IMG_HEIGHT = 160 #224
IMG_WIDTH = 160 #224
IMG_SIZE = 160
image_count = len(test_dir)
print(image_count)
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
AUTOTUNE = tf.data.experimental.AUTOTUNE

2480


To actually get the Image, we need to parse every bit of the image from the image file, using the function:
parse_data_without_augmentation on the Dataset of filename we have -> list_ds.

num_parallel_calls: (Optional.) If specified, the implementation creates a threadpool, which is used to fetch inputs from cycle elements asynchronously and in parallel. The default behavior is to fetch inputs from cycle elements synchronously with no parallelism. If the value tf.data.experimental.AUTOTUNE is used, then the number of parallel calls is set dynamically based on available CPU.

In [0]:
image_dataset = dataset_of_filenames.map(lambda x: parse_data_without_augmentation(x, IMAGE_SIZE), num_parallel_calls=AUTOTUNE)

In [226]:
image_dataset

<ParallelMapDataset shapes: (160, 160, 3), types: tf.float32>

Some things we can do to Dataset object:
- Apply transformation
- Concatenate
- Enumerate (Create tuples of (index, element) with specified index starting point)
- Filter


Since we cannot index a TF Dataset object, we can turn the Dataset into List form using the code below

In [0]:
img_ds_list = list(image_dataset.as_numpy_iterator())

In [228]:
img_ds_list[0]

array([[[ 0.01301964,  0.01301964,  0.01301964],
        [-0.4524706 , -0.4524706 , -0.4524706 ],
        [-0.69294125, -0.69294125, -0.69294125],
        ...,
        [-0.74313724, -0.74313724, -0.74313724],
        [-0.7359217 , -0.7359217 , -0.7359217 ],
        [-0.5748241 , -0.5748241 , -0.5748241 ]],

       [[-0.60313725, -0.60313725, -0.60313725],
        [-0.8778039 , -0.8778039 , -0.8778039 ],
        [-0.9580392 , -0.9580392 , -0.9580392 ],
        ...,
        [-0.9615686 , -0.9615686 , -0.9615686 ],
        [-0.9615686 , -0.9615686 , -0.9615686 ],
        [-0.9440785 , -0.9440785 , -0.9440785 ]],

       [[-0.827451  , -0.827451  , -0.827451  ],
        [-0.9611765 , -0.9611765 , -0.9611765 ],
        [-0.9843137 , -0.9843137 , -0.9843137 ],
        ...,
        [-0.9764706 , -0.9764706 , -0.9764706 ],
        [-0.9764706 , -0.9764706 , -0.9764706 ],
        [-0.9764706 , -0.9764706 , -0.9764706 ]],

       ...,

       [[-0.9403922 , -0.9403922 , -0.9403922 ],
        [-0

In [229]:
# The Images
for image in image_dataset:
  print(image)

Streaming output truncated to the last 5000 lines.
[[[-1.         -1.         -1.        ]
  [-1.         -1.         -1.        ]
  [-1.         -1.         -1.        ]
  ...
  [-1.         -1.         -1.        ]
  [-1.         -1.         -1.        ]
  [-1.         -1.         -1.        ]]

 [[-1.         -1.         -1.        ]
  [-1.         -1.         -1.        ]
  [-1.         -1.         -1.        ]
  ...
  [-0.82823527 -0.82823527 -0.82823527]
  [-0.8211763  -0.8211763  -0.8211763 ]
  [-0.82823527 -0.82823527 -0.82823527]]

 [[-1.         -1.         -1.        ]
  [-1.         -1.         -1.        ]
  [-1.         -1.         -1.        ]
  ...
  [-0.86470586 -0.86470586 -0.86470586]
  [-0.8635292  -0.8635292  -0.8635292 ]
  [-0.8666667  -0.8666667  -0.8666667 ]]

 ...

 [[-1.         -1.         -1.        ]
  [-1.         -1.         -1.        ]
  [-1.         -1.         -1.        ]
  ...
  [-1.         -1.         -1.        ]
  [-1.         -1.         -1.   

Splitting Dataset into Training, Testing & Validation 

In [230]:
total = len(sample_labels)
print("Total Data To Use: "+str(total))

Total Data To Use: 2480


In [231]:
# Note that we only have 100 images
train_size = int(0.7*total) #70
test_size = int(0.15*total) #15
val_size = int(0.15*total) #15
print(train_size)
print(test_size)
print(val_size)

1736
372
372


In [0]:
# Splitting Image Data
train_img = image_dataset.take(train_size)
test_img = image_dataset.skip(train_size)
val_img = test_img.skip(val_size)
test_img = test_img.take(test_size)

In [233]:
train_img

<TakeDataset shapes: (160, 160, 3), types: tf.float32>

In [234]:
train_lab = labels[0:train_size]
print(len(train_lab))
test_lab = labels[train_size:(train_size+test_size)]
print(len(test_lab))
val_lab = labels[(train_size+test_size):]
print(len(val_lab))

1736
372
372


In [235]:
for img in train_img:
  print(img)

Streaming output truncated to the last 5000 lines.
[[[-0.7944314  -0.7944314  -0.7944314 ]
  [-0.808549   -0.808549   -0.808549  ]
  [-0.8172549  -0.8172549  -0.8172549 ]
  ...
  [ 0.5686275   0.5686275   0.5686275 ]
  [ 0.5814125   0.5814125   0.5814125 ]
  [ 0.69262743  0.69262743  0.69262743]]

 [[-0.8117647  -0.8117647  -0.8117647 ]
  [-0.81819606 -0.81819606 -0.81819606]
  [-0.82000005 -0.82000005 -0.82000005]
  ...
  [ 0.5909804   0.5909804   0.5909804 ]
  [ 0.52956843  0.52956843  0.52956843]
  [ 0.6418037   0.6418037   0.6418037 ]]

 [[-0.8039216  -0.8039216  -0.8039216 ]
  [-0.81960785 -0.81960785 -0.81960785]
  [-0.8215686  -0.8215686  -0.8215686 ]
  ...
  [ 0.57254905  0.57254905  0.57254905]
  [ 0.56156844  0.56156844  0.56156844]
  [ 0.59764665  0.59764665  0.59764665]]

 ...

 [[-0.7501961  -0.7501961  -0.7501961 ]
  [-0.7556863  -0.7556863  -0.7556863 ]
  [-0.75490195 -0.75490195 -0.75490195]
  ...
  [-0.61764705 -0.61764705 -0.61764705]
  [-0.530588   -0.530588   -0.530

In [0]:
def show_batch(image_batch, num_elements): #, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(num_elements):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      #plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('off')

In [0]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  # This is a small dataset, only load it once, and keep it in memory.
  # use `.cache(filename)` to cache preprocessing work for datasets that don't
  # fit in memory.
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()

  #ds = ds.shuffle(buffer_size=shuffle_buffer_size)

  # Repeat forever
  #ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds

In [238]:
BATCH_SIZE = train_size
print("Train size: "+str(BATCH_SIZE))
train_img_batch = prepare_for_training(train_img)
BATCH_SIZE = test_size
print("Test size: "+str(BATCH_SIZE))
test_img_batch = prepare_for_training(test_img)
BATCH_SIZE = val_size
print("Val size: "+str(BATCH_SIZE))
val_img_batch = prepare_for_training(val_img)

Train size: 1736
Test size: 372
Val size: 372


In [0]:
#for img in train_img:
#  print(img)

In [0]:
#for img in train_img_batch:
#  print(img)

In [241]:
train_img_batch

<PrefetchDataset shapes: (None, 160, 160, 3), types: tf.float32>

In [0]:
train_batch = next(iter(train_img_batch))
test_batch = next(iter(test_img_batch))
val_batch = next(iter(val_img_batch))

In [243]:
print(train_batch.shape)
print(test_batch.shape)
print(val_batch.shape)

(1736, 160, 160, 3)
(372, 160, 160, 3)
(372, 160, 160, 3)


In [0]:
#show_batch(val_batch.numpy(), val_size)

## Feature Extraction Using Pre-trained MobileNetV2 Model

Often, it is better to get features using a pre-trained embedding model that have been trained with millions of images, instead of training our own embedding. We will use MobileNetV2.

In [0]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

In [246]:
print(IMG_SIZE)

160


In [0]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

Extracting features for train, test, and validation.

In [0]:
train_features = base_model(train_batch)
test_features = base_model(test_batch)
val_features = base_model(val_batch)

In [249]:
print(train_features.shape)
print(test_features.shape)
print(val_features.shape)

(1736, 5, 5, 1280)
(372, 5, 5, 1280)
(372, 5, 5, 1280)


We need to convert the numpy version of these features, to Input Tensors, before inputting it to the model.

In [0]:
# fnp = features numpy
train_fnp = train_features.numpy()
test_fnp = test_features.numpy()
val_fnp = val_features.numpy()

In [0]:
train_input = tf.convert_to_tensor(train_fnp, dtype=tf.float32)
test_input = tf.convert_to_tensor(test_fnp, dtype=tf.float32)
val_input = tf.convert_to_tensor(val_fnp, dtype=tf.float32)

In [252]:
test_input.shape

TensorShape([372, 5, 5, 1280])

Another Way:

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [254]:
train_batch_ave = global_average_layer(train_features)
print(train_batch_ave.shape)

(1736, 1280)


In [255]:
test_batch_ave = global_average_layer(test_features)
print(test_batch_ave.shape)

(372, 1280)


In [256]:
val_batch_ave = global_average_layer(val_features)
print(val_batch_ave.shape)

(372, 1280)


In [0]:
prediction_layer = tf.keras.layers.Dense(256) #(2) #256

In [258]:
train_pred_batch = prediction_layer(train_batch_ave)
print(train_pred_batch.shape)

(1736, 256)


In [259]:
test_pred_batch = prediction_layer(test_batch_ave)
print(test_pred_batch.shape)

(372, 256)


In [260]:
val_pred_batch = prediction_layer(val_batch_ave)
print(val_pred_batch.shape)

(372, 256)


In [0]:
train_input = train_pred_batch.numpy()

In [262]:
train_input.shape

(1736, 256)

In [263]:
test_input = test_pred_batch.numpy()
test_input.shape

(372, 256)

In [264]:
val_input = val_pred_batch.numpy()
val_input.shape

(372, 256)

# Pre-processing the Labels:


Since the labels are texts, we need to create an embedding or number / vector representations from it. 

We will use:
- LabelEncoder -> assigning integer 0 to number of unique categorical variable to each categorical variable. 
- OneHotEncoding

In [265]:
num_of_labels = len(labels)
print(num_of_labels)

2480


In [266]:
unique_labels = sample_labels['Finding Labels'].unique().tolist()
print(unique_labels[:5])
num_of_unique = len(unique_labels)
num_of_unique

['effusion', 'infiltration', 'consolidation', 'pleural_thickening', 'no finding']


9

In [267]:
labels[:5]

['effusion', 'effusion', 'infiltration', 'effusion', 'consolidation']

In [0]:
from sklearn import preprocessing

Creating the Label Encoder object from ALL of our Labels

In [0]:
le = preprocessing.LabelEncoder()

In [270]:
le.fit(labels)

LabelEncoder()

Encode train, test, and validation labels with Integer Values.

In [0]:
train_label_value = le.transform(train_lab)
test_label_value = le.transform(test_lab)
val_label_value = le.transform(val_lab)

In [272]:
len(train_lab)

1736

In [0]:
label_to_value = pd.DataFrame(data=train_lab[:10], columns=['Original Label'])
label_to_value['Label Value'] = train_label_value[:10]

In [274]:
all_labels_value = le.transform(labels)
len(all_labels_value)

2480

Below is the Example of Encoding Validation Label to their label values. 

Notice that all 'No Finding' becomes the value 24

In [275]:
label_to_value

,Original Label,Label Value
0,effusion,2
1,effusion,2
2,infiltration,3
3,effusion,2
4,consolidation,1
5,pleural_thickening,7
6,infiltration,3
7,infiltration,3
8,effusion,2
9,no finding,5


In [0]:
train_one_hot = tf.one_hot(train_label_value, num_of_unique, on_value=1, off_value=0)
test_one_hot = tf.one_hot(test_label_value, num_of_unique, on_value=1, off_value=0)
val_one_hot = tf.one_hot(val_label_value, num_of_unique, on_value=1, off_value=0)

In [0]:
all_labels_one_hot = tf.one_hot(all_labels_value, num_of_unique, on_value=1, off_value=0)

In [0]:
label_to_value['One Hot'] = train_one_hot[:10]

In [279]:
label_to_value

,Original Label,Label Value,One Hot
0,effusion,2,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
1,effusion,2,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
2,infiltration,3,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
3,effusion,2,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
4,consolidation,1,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
5,pleural_thickening,7,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
6,infiltration,3,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
7,infiltration,3,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
8,effusion,2,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
9,no finding,5,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."


In [0]:
all_labels_df = pd.DataFrame(data=labels, columns=['Label'])
all_labels_df['Value'] = all_labels_value
all_labels_df['One Hot'] = all_labels_one_hot

In [281]:
all_labels_df

,Label,Value,One Hot
0,effusion,2,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
1,effusion,2,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
2,infiltration,3,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
3,effusion,2,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
4,consolidation,1,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
...,...,...,...
2475,pneumothorax,8,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
2476,effusion,2,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
2477,pleural_thickening,7,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
2478,atelectasis,0,"(tf.Tensor(1, shape=(), dtype=int32), tf.Tenso..."


# Creating the CT Scan Image Classification Model

In [0]:
from tensorflow.keras.models import Sequential

Additional Metrics Functions on top of Accuracy

In [0]:
def recall_m(y_true, y_pred):
        true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
        possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
        predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+tf.keras.backend.epsilon()))

Creating the Model:

Base Model + Our Model Combined

In [0]:
model_combined = keras.Sequential([
    base_model,
    global_average_layer,                        
    keras.Input(shape=(256)), 
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(num_of_unique, activation='softmax')
])

In [0]:
model_combined.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
input_2 (InputLayer)         multiple                  0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               163968    
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 1161      
Total params: 2,423,113
Trainable params: 2,389,001
Non-trainable params: 34,112
_________________________________________________________________


In [0]:
model_combined.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy', precision_m, recall_m, f1_m])

In [0]:
history_comb = model_combined.fit(train_batch, train_one_hot, epochs=100, validation_data = (val_batch, val_one_hot))

Epoch 1/100
55/55 [==============================] - 98s 2s/step - loss: 2.0558 - accuracy: 0.2730 - precision_m: 0.3246 - recall_m: 0.0284 - f1_m: 0.0510 - val_loss: 3.3100 - val_accuracy: 0.2339 - val_precision_m: 0.2799 - val_recall_m: 0.1453 - val_f1_m: 0.1905
Epoch 2/100
55/55 [==============================] - 96s 2s/step - loss: 1.8562 - accuracy: 0.3370 - precision_m: 0.5094 - recall_m: 0.0989 - f1_m: 0.1606 - val_loss: 6.0843 - val_accuracy: 0.1855 - val_precision_m: 0.1922 - val_recall_m: 0.1797 - val_f1_m: 0.1857
Epoch 3/100
55/55 [==============================] - 95s 2s/step - loss: 1.7315 - accuracy: 0.3980 - precision_m: 0.6031 - recall_m: 0.1335 - f1_m: 0.2150 - val_loss: 3.9740 - val_accuracy: 0.1210 - val_precision_m: 0.1007 - val_recall_m: 0.0917 - val_f1_m: 0.0959
Epoch 4/100
55/55 [==============================] - 94s 2s/step - loss: 1.5880 - accuracy: 0.4326 - precision_m: 0.6436 - recall_m: 0.2210 - f1_m: 0.3251 - val_loss: 7.2571 - val_accuracy: 0.1102 - val_pr

KeyboardInterrupt: ignored

In [0]:
model_combined_2 = keras.Sequential([
    base_model,
    global_average_layer,                        
    #keras.Input(shape=(256)), 
    prediction_layer, 
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(rate=0.7),
    keras.layers.Dense(num_of_unique, activation='softmax')
])

In [0]:
model_combined_2.summary()
model_combined_2.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy', precision_m, recall_m, f1_m])
history_comb = model_combined_2.fit(train_batch, train_one_hot, epochs=100, validation_data = (val_batch, val_one_hot))

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               327936    
_________________________________________________________________
dense_16 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 9)                 1161      
Total params: 2,619,977
Trainable params: 2,585,865
Non-trainable params: 34,112
_______________________________________

KeyboardInterrupt: ignored

In [0]:
model_combined_3 = keras.Sequential([
    base_model,
    global_average_layer,                        
    #keras.Input(shape=(256)), 
    #prediction_layer, 
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(rate=0.5),
    keras.layers.Dense(num_of_unique, activation='softmax')
])

In [0]:
# Using Early Stopping:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=2)

In [0]:
model_combined_3.summary()
model_combined_3.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy', precision_m, recall_m, f1_m])
history_comb = model_combined_3.fit(train_batch, train_one_hot, epochs=100, validation_data = (val_batch, val_one_hot))

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 256)               327936    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 9)                 2313      
Total params: 2,588,233
Trainable params: 2,554,121
Non-trainable params: 34,112
_________________________________________________________________
Epoch 1/100
55/55 [==============================] - 93s 2s/step - loss: 1.8702 - accuracy: 0.3923 - prec

KeyboardInterrupt: ignored

In [0]:
model_combined_4_2 = keras.Sequential([
    base_model,
    global_average_layer,                        
    #keras.Input(shape=(256)), 
    #prediction_layer, 
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(rate=0.7),
    keras.layers.Dense(num_of_unique, activation='softmax')
])

In [0]:
# Using Early Stopping:
#callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

In [285]:
model_combined_4_2.summary()
model_combined_4_2.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy', precision_m, recall_m, f1_m])
history_comb = model_combined_4_2.fit(train_batch, train_one_hot, epochs=25, validation_data = (val_batch, val_one_hot))

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 256)               327936    
_________________________________________________________________
dropout_11 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 9)                 2313      
Total params: 2,588,233
Trainable params: 2,554,121
Non-trainable params: 34,112
_________________________________________________________________
Epoch 1/25
55/55 [==============================] - 146s 3s/step - loss: 2.1945 - accuracy: 0.2275 - pre

KeyboardInterrupt: ignored

In [286]:
evaluation_combined_4_1 = model_combined_4_1.evaluate(test_batch, test_one_hot)

12/12 [==============================] - 3s 275ms/step - loss: 10.2151 - accuracy: 0.1855 - precision_m: 0.1881 - recall_m: 0.1865 - f1_m: 0.1873


In [0]:
history_comb = model_combined_4_1.fit(train_batch, train_one_hot, epochs=2, callbacks=[callback], validation_data = (val_batch, val_one_hot))

Epoch 1/2
55/55 [==============================] - 93s 2s/step - loss: 0.2566 - accuracy: 0.9309 - precision_m: 0.9402 - recall_m: 0.9068 - f1_m: 0.9229 - val_loss: 12.9366 - val_accuracy: 0.2070 - val_precision_m: 0.2096 - val_recall_m: 0.2083 - val_f1_m: 0.2090
Epoch 2/2
55/55 [==============================] - 94s 2s/step - loss: 0.3304 - accuracy: 0.9119 - precision_m: 0.9243 - recall_m: 0.8892 - f1_m: 0.9061 - val_loss: 18.4325 - val_accuracy: 0.2473 - val_precision_m: 0.2490 - val_recall_m: 0.2490 - val_f1_m: 0.2490


In [0]:
history_comb = model_combined_4_1.fit(train_batch, train_one_hot, epochs=1, callbacks=[callback], validation_data = (val_batch, val_one_hot))

55/55 [==============================] - 93s 2s/step - loss: 0.2155 - accuracy: 0.9441 - precision_m: 0.9575 - recall_m: 0.9312 - f1_m: 0.9440 - val_loss: 22.3724 - val_accuracy: 0.1183 - val_precision_m: 0.1177 - val_recall_m: 0.1177 - val_f1_m: 0.1177


In [0]:
history_comb = model_combined_4_1.fit(train_batch, train_one_hot, epochs=1, callbacks=[callback], validation_data = (val_batch, val_one_hot))

55/55 [==============================] - 93s 2s/step - loss: 0.2038 - accuracy: 0.9470 - precision_m: 0.9532 - recall_m: 0.9318 - f1_m: 0.9422 - val_loss: 22.8210 - val_accuracy: 0.1263 - val_precision_m: 0.1255 - val_recall_m: 0.1255 - val_f1_m: 0.1255


In [0]:
history_comb = model_combined_4_1.fit(train_batch, train_one_hot, epochs=1, callbacks=[callback], validation_data = (val_batch, val_one_hot))

55/55 [==============================] - 94s 2s/step - loss: 0.3721 - accuracy: 0.8969 - precision_m: 0.9176 - recall_m: 0.8642 - f1_m: 0.8895 - val_loss: 27.1765 - val_accuracy: 0.1371 - val_precision_m: 0.1359 - val_recall_m: 0.1359 - val_f1_m: 0.1359


In [0]:
model_1 = keras.Sequential([
    keras.Input(shape=(256)), 
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(num_of_unique, activation='softmax')
])

In [0]:
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 9)                 1161      
Total params: 34,057
Trainable params: 34,057
Non-trainable params: 0
_________________________________________________________________


In [0]:
model_1.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy', precision_m, recall_m, f1_m])

In [0]:
history = model_1.fit(train_input, train_one_hot, epochs=25, validation_data = (val_input, val_one_hot))

Epoch 1/25
55/55 [==============================] - 0s 7ms/step - loss: 2.1282 - accuracy: 0.2483 - precision_m: 0.2310 - recall_m: 0.0216 - f1_m: 0.0382 - val_loss: 2.0446 - val_accuracy: 0.3226 - val_precision_m: 0.3806 - val_recall_m: 0.0406 - val_f1_m: 0.0716
Epoch 2/25
55/55 [==============================] - 0s 2ms/step - loss: 1.9225 - accuracy: 0.3209 - precision_m: 0.3923 - recall_m: 0.0420 - f1_m: 0.0736 - val_loss: 2.0357 - val_accuracy: 0.2581 - val_precision_m: 0.1667 - val_recall_m: 0.0104 - val_f1_m: 0.0196
Epoch 3/25
55/55 [==============================] - 0s 2ms/step - loss: 1.8386 - accuracy: 0.3520 - precision_m: 0.5199 - recall_m: 0.0477 - f1_m: 0.0841 - val_loss: 2.0546 - val_accuracy: 0.3306 - val_precision_m: 0.3839 - val_recall_m: 0.0974 - val_f1_m: 0.1544
Epoch 4/25
55/55 [==============================] - 0s 2ms/step - loss: 1.7742 - accuracy: 0.3773 - precision_m: 0.5928 - recall_m: 0.0733 - f1_m: 0.1248 - val_loss: 2.0270 - val_accuracy: 0.2876 - val_precis

In [0]:
model_2 = keras.Sequential([
    keras.Input(shape=(256)),     
    keras.layers.Dense(500, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.45)),
    keras.layers.Dense(320, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.11)),     
    keras.layers.Dense(300,kernel_regularizer=tf.keras.regularizers.l2(0.07)),     
    keras.layers.Dropout(rate=0.001),
    keras.layers.Dense(180),    
    #keras.layers.Dense(10, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.07)),                 

    keras.layers.Dense(num_of_unique, activation='softmax')
])

model_2.summary()

model_2.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), metrics=['accuracy', precision_m, recall_m, f1_m])

#history = model_2.fit(train_input, train_one_hot, epochs=25, validation_data = (val_input, val_one_hot)) # epochs=300

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 500)               128500    
_________________________________________________________________
dense_6 (Dense)              (None, 320)               160320    
_________________________________________________________________
dense_7 (Dense)              (None, 300)               96300     
_________________________________________________________________
dropout (Dropout)            (None, 300)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 180)               54180     
_________________________________________________________________
dense_9 (Dense)              (None, 9)                 1629      
Total params: 440,929
Trainable params: 440,929
Non-trainable params: 0
________________________________________________

In [0]:
# Using Early Stopping:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=2)

In [0]:
# Using Early Stopping:
callback_2 = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=20)

In [0]:
history = model_2.fit(train_input, train_one_hot, epochs=25, callbacks=[callback_2], validation_data = (val_input, val_one_hot)) # epochs=300

Epoch 1/25
55/55 [==============================] - 2s 42ms/step - loss: 144.5013 - accuracy: 0.2535 - precision_m: 0.0182 - recall_m: 5.6818e-04 - f1_m: 0.0011 - val_loss: 84.3837 - val_accuracy: 0.2634 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 2/25
55/55 [==============================] - 1s 26ms/step - loss: 53.0032 - accuracy: 0.2776 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - f1_m: 0.0000e+00 - val_loss: 29.5856 - val_accuracy: 0.2903 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 3/25
55/55 [==============================] - 2s 37ms/step - loss: 18.3553 - accuracy: 0.2690 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - f1_m: 0.0000e+00 - val_loss: 10.2951 - val_accuracy: 0.2500 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 4/25
55/55 [==============================] - 2s 38ms/step - loss: 6.7274 - accuracy: 0.2535 - precision_m: 0.0000e+00 - recall_

In [0]:
model_3 = keras.Sequential([
    keras.Input(shape=(256)), 
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(rate=0.7),
    keras.layers.Dense(num_of_unique, activation='softmax'),
])

model_3.summary()
model_3.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy', precision_m, recall_m, f1_m])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 9)                 1161      
Total params: 34,057
Trainable params: 34,057
Non-trainable params: 0
_________________________________________________________________


In [0]:
history_3 = model_3.fit(train_input, train_one_hot, epochs=25, validation_data = (val_input, val_one_hot))

Epoch 1/25
55/55 [==============================] - 0s 6ms/step - loss: 2.4301 - accuracy: 0.2068 - precision_m: 0.1773 - recall_m: 0.0267 - f1_m: 0.0421 - val_loss: 2.0525 - val_accuracy: 0.2608 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 2/25
55/55 [==============================] - 0s 3ms/step - loss: 2.1121 - accuracy: 0.2166 - precision_m: 0.1485 - recall_m: 0.0074 - f1_m: 0.0140 - val_loss: 2.0469 - val_accuracy: 0.2876 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 3/25
55/55 [==============================] - 0s 3ms/step - loss: 2.0957 - accuracy: 0.2465 - precision_m: 0.0803 - recall_m: 0.0051 - f1_m: 0.0095 - val_loss: 2.0281 - val_accuracy: 0.3011 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 4/25
55/55 [==============================] - 0s 2ms/step - loss: 2.0379 - accuracy: 0.2713 - precision_m: 0.3015 - recall_m: 0.0176 - f1_m: 0.0326 - val_loss: 2.0208

In [0]:
model_4 = keras.Sequential([
    keras.Input(shape=(256)), 
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(rate=0.7),
    keras.layers.Dense(num_of_unique, activation='softmax'),
])

model_4.summary()
model_4.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy', precision_m, recall_m, f1_m])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 9)                 1161      
Total params: 34,057
Trainable params: 34,057
Non-trainable params: 0
_________________________________________________________________


In [0]:
history_4 = model_4.fit(train_input, train_one_hot, epochs=300, callbacks=[callback], validation_data = (val_input, val_one_hot))

Epoch 1/300
55/55 [==============================] - 0s 5ms/step - loss: 2.4092 - accuracy: 0.1849 - precision_m: 0.1119 - recall_m: 0.0182 - f1_m: 0.0282 - val_loss: 2.0668 - val_accuracy: 0.2823 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 2/300
55/55 [==============================] - 0s 2ms/step - loss: 2.1111 - accuracy: 0.2287 - precision_m: 0.1788 - recall_m: 0.0102 - f1_m: 0.0190 - val_loss: 2.0637 - val_accuracy: 0.2957 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 3/300
55/55 [==============================] - 0s 2ms/step - loss: 2.0864 - accuracy: 0.2494 - precision_m: 0.1303 - recall_m: 0.0057 - f1_m: 0.0108 - val_loss: 2.0320 - val_accuracy: 0.2930 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 4/300
55/55 [==============================] - 0s 2ms/step - loss: 2.0612 - accuracy: 0.2667 - precision_m: 0.1818 - recall_m: 0.0080 - f1_m: 0.0151 - val_loss: 2.

In [0]:
model_5 = keras.Sequential([
    keras.Input(shape=(256)), 
    keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.25)),
    keras.layers.Dropout(rate=0.7),
    keras.layers.Dense(num_of_unique, activation='softmax'),
])

model_5.summary()
model_5.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy', precision_m, recall_m, f1_m])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 9)                 1161      
Total params: 34,057
Trainable params: 34,057
Non-trainable params: 0
_________________________________________________________________


In [0]:
history_5 = model_5.fit(train_input, train_one_hot, epochs=300, validation_data = (val_input, val_one_hot))

Epoch 1/300
55/55 [==============================] - 0s 5ms/step - loss: 28.2297 - accuracy: 0.1849 - precision_m: 0.1806 - recall_m: 0.0352 - f1_m: 0.0530 - val_loss: 14.5763 - val_accuracy: 0.2608 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 2/300
55/55 [==============================] - 0s 2ms/step - loss: 8.8504 - accuracy: 0.2379 - precision_m: 0.0800 - recall_m: 0.0034 - f1_m: 0.0064 - val_loss: 4.9688 - val_accuracy: 0.2984 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 3/300
55/55 [==============================] - 0s 2ms/step - loss: 3.6185 - accuracy: 0.2494 - precision_m: 0.1545 - recall_m: 0.0057 - f1_m: 0.0109 - val_loss: 2.7400 - val_accuracy: 0.2849 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 4/300
55/55 [==============================] - 0s 2ms/step - loss: 2.4368 - accuracy: 0.2546 - precision_m: 0.0864 - recall_m: 0.0034 - f1_m: 0.0065 - val_loss: 

In [0]:
#history = model_1.fit(train_input, train_one_hot, epochs=20, validation_data = (val_input, val_one_hot))

EVALUATION

In [0]:
evaluation = model_1.evaluate(test_input, test_one_hot)

12/12 [==============================] - 0s 2ms/step - loss: 2.6085 - accuracy: 0.2204 - precision_m: 0.2853 - recall_m: 0.1307 - f1_m: 0.1747


In [0]:
def create_evaluation_table(model, x_input, actual_label):
  test_pred = model_1.predict(x_input)
  argmarx_ = [np.argmax(pred) for pred in test_pred]
  pred_labels = [all_labels_df[all_labels_df.Value == val].reset_index().iloc[0]['Label'] for val in argmarx_]
  compare = pd.DataFrame(data=actual_label, columns=['Actual'])
  compare['Predicted'] = pred_labels
  return compare

In [0]:
test_eval_table = create_evaluation_table(model_1, test_input, test_lab)

In [0]:
#test_eval_table

In [0]:
#test_eval_table[0:50]

In [0]:
evaluation_2 = model_2.evaluate(test_input, test_one_hot)

12/12 [==============================] - 0s 3ms/step - loss: 2.1000 - accuracy: 0.1935 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - f1_m: 0.0000e+00


In [0]:
evaluation_2_table = create_evaluation_table(model_2, val_input, val_lab)

In [0]:
#evaluation_2_table[0:60]

In [0]:
evaluation_3 = model_3.evaluate(test_input, test_one_hot)

12/12 [==============================] - 0s 1ms/step - loss: 2.0293 - accuracy: 0.2903 - precision_m: 0.2639 - recall_m: 0.0130 - f1_m: 0.0245


In [0]:
evaluation_4 = model_4.evaluate(test_input, test_one_hot)

12/12 [==============================] - 0s 1ms/step - loss: 2.0542 - accuracy: 0.2688 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - f1_m: 0.0000e+00


In [0]:
evaluation_5 = model_5.evaluate(test_input, test_one_hot)

12/12 [==============================] - 0s 1ms/step - loss: 2.0894 - accuracy: 0.1935 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - f1_m: 0.0000e+00


In [0]:
val_eval = model_1.evaluate(val_input, val_one_hot)

12/12 [==============================] - 0s 1ms/step - loss: 2.5787 - accuracy: 0.2285 - precision_m: 0.2301 - recall_m: 0.1214 - f1_m: 0.1579


In [0]:
val_eval_table = create_evaluation_table(model_1, val_input, val_lab)

In [0]:
val_eval_table

,Actual,Predicted
0,infiltration,infiltration
1,infiltration,consolidation
2,no finding,pneumothorax
3,pleural_thickening,infiltration
4,pneumothorax,effusion
...,...,...
367,nodule,infiltration
368,effusion,effusion
369,infiltration,infiltration
370,effusion,no finding


In [0]:
train_eval = model_1.evaluate(train_input, train_one_hot)

55/55 [==============================] - 0s 1ms/step - loss: 0.5640 - accuracy: 0.8825 - precision_m: 0.9765 - recall_m: 0.7222 - f1_m: 0.8283


In [0]:
train_eval_table = create_evaluation_table(model_1, train_input, train_lab)

In [0]:
train_eval_table

,Actual,Predicted
0,pneumothorax,pneumothorax
1,no finding,no finding
2,infiltration,infiltration
3,pleural_thickening,pleural_thickening
4,pleural_thickening,pleural_thickening
...,...,...
1731,effusion,effusion
1732,pneumothorax,pneumothorax
1733,mass,infiltration
1734,infiltration,infiltration
